# LBP Disc with Irradiated EOS and Zhu Opacity

Using IrradiatedEOS to compute temperature self-consistently from irradiation + viscous heating

In [12]:
# Imports
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from DiscEvolution.grid import Grid
from DiscEvolution.star import SimpleStar
from DiscEvolution.eos import IrradiatedEOS
from DiscEvolution.disc import AccretionDisc
from DiscEvolution.viscous_evolution import ViscousEvolutionFV
from DiscEvolution.driver import PlanetDiscDriver
from DiscEvolution.io import Event_Controller

In [19]:
# Physical parameters
Mstar = 1.0 * u.solMass
Rstar = 1.0 * u.solRad
Teff = 5770 * u.K  # Effective temperature
mu = 2.33  # Mean molecular weight

# Disc parameters
M_disk_0 = 0.05 * u.solMass
Rc = 1.0 * u.AU  # LBP characteristic radius
alpha_visc = 0.01  # Viscous alpha parameter (increased from 1e-3)
t_final = 1.0 * u.Myr

# Grid
R_in = 0.1  # AU
R_out = 3.0  # AU (reduce to avoid cool outer disc dominating CFL)
Ncells = 100

print(f"Setup parameters:")
print(f"  Star: M = {Mstar.to(u.solMass)}, R = {Rstar.to(u.solRad)}, Teff = {Teff}")
print(f"  Disc: M = {M_disk_0.to(u.solMass)}, Rc = {Rc.to(u.AU)}")
print(f"  Grid: {R_in}-{R_out} AU, {Ncells} cells")

Setup parameters:
  Star: M = 1.0 solMass, R = 1.0 solRad, Teff = 5770.0 K
  Disc: M = 0.05 solMass, Rc = 1.0 AU
  Grid: 0.1-3.0 AU, 100 cells


In [20]:
# LBP surface density profile
def Sigma_LBP(r):
    """LBP profile: Sigma(r) = (M_disk / (2*pi*Rc*r)) * exp(-r/Rc)"""
    val = (M_disk_0)/(2 * np.pi * Rc * r) * np.exp(-r / Rc)
    return val.cgs

# Create grid and star
grid = Grid(R_in, R_out, Ncells, spacing='log')
star = SimpleStar(M=1.0)  # Unitless

# Create irradiated EOS with Zhu2012 opacity (default)
# IrradiatedEOS computes temperature self-consistently from:
# - Irradiation from the star
# - Viscous heating from accretion
# - Zhu2012 opacity (default)
eos = IrradiatedEOS(
    star,
    alpha_t=alpha_visc,
    Tc=10,        # External (nebular) temperature in K
    Tmax=2000,    # Maximum temperature cap (increased)
    mu=mu,
    gamma=1.4,
    accrete=True  # Include viscous heating
)
eos.set_grid(grid)

# Create initial surface density from LBP profile
Sigma_initial = np.array([Sigma_LBP(r * u.AU).value for r in grid.Rc])

# Create disc
disc = AccretionDisc(grid, star, eos, Sigma=Sigma_initial)
disc._planetesimal = False

# Update EOS with initial Sigma to compute temperatures
eos.update(0, Sigma_initial, amax=1e-5)

# Verify setup
print(f"\nDisc setup:")
print(f"  Grid: {disc.Ncells} cells from {grid.Rc[0]:.2f} to {grid.Rc[-1]:.2f} AU")
print(f"  Disc mass: {disc.Mtot()/const.M_sun.to(u.g).value:.3e} M_sun")
print(f"  \nTemperature profile (from irradiated EOS + Zhu opacity):")
idx_1AU = np.argmin(np.abs(disc.R - 1.0))
print(f"    T at 1 AU: {disc.T[idx_1AU]:.1f} K")
print(f"    T at 5 AU: {disc.T[np.argmin(np.abs(disc.R - 5.0))]:.1f} K")
print(f"    T at 10 AU: {disc.T[-1]:.1f} K")
print(f"  \nViscosity profile:")
print(f"    nu at 1 AU: {disc.nu[idx_1AU]:.2e} cm²/s")
print(f"    nu at 10 AU: {disc.nu[-1]:.2e} cm²/s")
print(f"    Max nu: {disc.nu.max():.2e} cm²/s")


Disc setup:
  Grid: 100 cells from 0.10 to 2.95 AU
  Disc mass: 4.276e-02 M_sun
  
Temperature profile (from irradiated EOS + Zhu opacity):
    T at 1 AU: 2000.0 K
    T at 5 AU: 1493.7 K
    T at 10 AU: 1493.7 K
  
Viscosity profile:
    nu at 1 AU: 7.96e-04 cm²/s
    nu at 10 AU: 3.04e-03 cm²/s
    Max nu: 3.04e-03 cm²/s


In [21]:
# Set up viscous evolution
visc_evol = ViscousEvolutionFV(tol=0.5, boundary='power_law', in_bound='Mdot')

# Create driver
driver = PlanetDiscDriver(disc, gas=visc_evol, t0=0.0)

# Check initial timestep
dt_max_initial = visc_evol.max_timestep(disc)
print(f"Initial timestep diagnostic:")
print(f"  dt_max: {dt_max_initial:.3e} seconds")
print(f"  dt_max: {dt_max_initial * (1*u.s).to(u.yr).value:.3e} years")
print(f"  dt_max: {dt_max_initial * (1*u.s).to(u.Myr).value:.3e} Myr")
print(f"  Viscosity range: {disc.nu.min():.2e} to {disc.nu.max():.2e} cm²/s")

# Estimate number of steps
t_final_sec = t_final.to(u.s).value
n_steps_estimate = t_final_sec / dt_max_initial
print(f"  Estimated steps to {t_final.to(u.Myr).value} Myr: {n_steps_estimate:.1e}")

Initial timestep diagnostic:
  dt_max: 3.822e-02 seconds
  dt_max: 1.211e-09 years
  dt_max: 1.211e-15 Myr
  Viscosity range: 2.61e-05 to 3.04e-03 cm²/s
  Estimated steps to 1.0 Myr: 8.3e+14


In [16]:
# Run evolution
output_times = np.linspace(0, t_final_sec, 11)  # Output at 0, 0.1, 0.2, ... 1.0 Myr
io = Event_Controller(save=output_times)

history = {'t': [], 'Mtot': [], 'dt': [], 'T_mean': []}

pbar = tqdm(total=t_final.to(u.yr).value, unit='yr', desc='Evolution')
last_t = 0.0
step_count = 0

while not io.finished():
    ti = io.next_event_time()
    
    while driver.t < ti:
        dt = driver(ti)
        step_count += 1
        
        pbar.update((driver.t - last_t) * (1*u.s).to(u.yr).value)
        last_t = driver.t
        
        if step_count % 1000 == 0:
            dt_yr = dt * (1*u.s).to(u.yr).value
            print(f"Step {step_count}: t = {driver.t * (1*u.s).to(u.yr).value:.2e} yr, dt = {dt_yr:.2e} yr")
    
    if io.check_event(driver.t, 'save'):
        eos.update(driver.t, disc.Sigma, amax=1e-5)
        dt_current = visc_evol.max_timestep(disc)
        history['t'].append(driver.t * (1*u.s).to(u.yr).value)
        history['Mtot'].append(disc.Mtot() * u.g.to(u.solMass))
        history['dt'].append(dt_current * (1*u.s).to(u.yr).value)
        history['T_mean'].append(disc.T.mean())
        io.pop_events(driver.t, 'save')

pbar.close()

print(f"\nEvolution complete!")
print(f"  Total steps: {step_count}")
print(f"  Final time: {driver.t * (1*u.s).to(u.Myr).value:.3f} Myr")
print(f"  Final mass: {history['Mtot'][-1]:.3e} M_sun")
print(f"  Mean temperature: {history['T_mean'][-1]:.1f} K")

Evolution:   0%|          | 0.0004397543649112585/1000000.0 [01:17<49118638:04:06, 176827.10s/yr]


Evolution:   0%|          | 0.0004397543649112585/1000000.0 [01:17<49118638:04:06, 176827.10s/yr]


Step 1000: t = 1.21e-05 yr, dt = 1.21e-08 yr


Evolution:   0%|          | 0.0004397543649112585/1000000.0 [01:17<49118638:04:06, 176827.10s/yr]


Step 1000: t = 1.21e-05 yr, dt = 1.21e-08 yr


...

Step 44000: t = 5.33e-04 yr, dt = 1.21e-08 yr


Step 45000: t = 5.45e-04 yr, dt = 1.21e-08 yr


Step 46000: t = 5.57e-04 yr, dt = 1.21e-08 yr


Step 47000: t = 5.69e-04 yr, dt = 1.21e-08 yr


KeyboardInterrupt: 

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Mass evolution
axes[0, 0].plot(history['t'], history['Mtot'])
axes[0, 0].set_xlabel('Time (yr)')
axes[0, 0].set_ylabel('Disc Mass (M_sun)')
axes[0, 0].set_title('Disc Mass Evolution')
axes[0, 0].grid()

# Temperature evolution
axes[0, 1].plot(history['t'], history['T_mean'])
axes[0, 1].set_xlabel('Time (yr)')
axes[0, 1].set_ylabel('Mean Temperature (K)')
axes[0, 1].set_title('Mean Temperature Evolution')
axes[0, 1].grid()

# Timestep evolution
axes[1, 0].semilogy(history['t'], history['dt'])
axes[1, 0].set_xlabel('Time (yr)')
axes[1, 0].set_ylabel('Timestep (yr)')
axes[1, 0].set_title('Adaptive Timestep Evolution')
axes[1, 0].grid()

# Final density and temperature profiles
ax2 = axes[1, 1]
ax3 = ax2.twinx()
ax2.semilogy(disc.R, disc.Sigma, 'b-', label='Sigma')
ax3.plot(disc.R, disc.T, 'r-', label='T')
ax2.set_xlabel('Radius (AU)')
ax2.set_ylabel('Surface Density (g/cm²)', color='b')
ax3.set_ylabel('Temperature (K)', color='r')
ax2.tick_params(axis='y', labelcolor='b')
ax3.tick_params(axis='y', labelcolor='r')
ax2.set_title('Final Profiles')
ax2.grid()

plt.tight_layout()
plt.show()